# Regular expressions

![title](img/3.png)

![title](img/4.png)

![title](img/5.png)

In [25]:
#regular expression python library
import re

In [27]:
 import pandas as pd

time_sentences = ["Monday: The doctor's appointment is at 2:45pm.", 
                  "Tuesday: The dentist's appointment is at 11:30 am.",
                  "Wednesday: At 7:00pm, there is a basketball game!",
                  "Thursday: Be back home by 11:15 pm at the latest.",
                  "Friday: Take the train at 08:10 am, arrive at 09:00am."]

df = pd.DataFrame(time_sentences, columns=['text'])
df

,text
0,Monday: The doctor's appointment is at 2:45pm.
1,Tuesday: The dentist's appointment is at 11:30...
2,"Wednesday: At 7:00pm, there is a basketball game!"
3,Thursday: Be back home by 11:15 pm at the latest.
4,"Friday: Take the train at 08:10 am, arrive at ..."


In [28]:
#digits
df['text'].str.findall(r'\d')

0                   [2, 4, 5]
1                [1, 1, 3, 0]
2                   [7, 0, 0]
3                [1, 1, 1, 5]
4    [0, 8, 1, 0, 0, 9, 0, 0]
Name: text, dtype: object

In [29]:
#words
df['text'].str.findall(r'\w')

0    [M, o, n, d, a, y, T, h, e, d, o, c, t, o, r, ...
1    [T, u, e, s, d, a, y, T, h, e, d, e, n, t, i, ...
2    [W, e, d, n, e, s, d, a, y, A, t, 7, 0, 0, p, ...
3    [T, h, u, r, s, d, a, y, B, e, b, a, c, k, h, ...
4    [F, r, i, d, a, y, T, a, k, e, t, h, e, t, r, ...
Name: text, dtype: object

In [30]:
#hours:minutes
df['text'].str.findall(r'(\d?\d):(\d\d)')

0               [(2, 45)]
1              [(11, 30)]
2               [(7, 00)]
3              [(11, 15)]
4    [(08, 10), (09, 00)]
Name: text, dtype: object

In [31]:
#replace 'some' day with 'this was a day name'
df['text'].str.replace(r'\w+day\b', 'This was a day name')

0    This was a day name: The doctor's appointment ...
1    This was a day name: The dentist's appointment...
2    This was a day name: At 7:00pm, there is a bas...
3    This was a day name: Be back home by 11:15 pm ...
4    This was a day name: Take the train at 08:10 a...
Name: text, dtype: object

In [32]:
#extract method to create a df
df['text'].str.extract(r'(\d?\d):(\d\d)')

,0,1
0,2,45
1,11,30
2,7,00
3,11,15
4,08,10


In [33]:
#extractall (match columns to show the repititions occured)
df['text'].str.extractall(r'((\d?\d):(\d\d) ?([ap]m))')

0   1   2   3
  match                      
0 0        2:45pm   2  45  pm
1 0      11:30 am  11  30  am
2 0        7:00pm   7  00  pm
3 0      11:15 pm  11  15  pm
4 0      08:10 am  08  10  am
  1       09:00am  09  00  am

![title](6.png)

![title](7.png)

![title](8.png)

## Assignment 1

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [16]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [17]:
dates_extracted = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))')

dates_extracted

,,origin,month,day,year
,match,,,,
14,0,5/24/1990,5,24,1990
15,0,1/25/2011,1,25,2011
17,0,10/13/1976,10,13,1976
24,0,07/25/1984,07,25,1984
30,0,03/31/1985,03,31,1985
34,0,5/12/2012,5,12,2012
38,0,7/27/1986,7,27,1986
43,0,4/13/2002,4,13,2002
45,0,2/15/1998,2,15,1998


In [19]:
index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))'))
dates_extracted

,,origin,month,day,year,3,4
,match,,,,,,
14,0,5/24/1990,5,24,1990,NaN,NaN
15,0,1/25/2011,1,25,2011,NaN,NaN
17,0,10/13/1976,10,13,1976,NaN,NaN
24,0,07/25/1984,07,25,1984,NaN,NaN
30,0,03/31/1985,03,31,1985,NaN,NaN
...,...,...,...,...,...,...,...
120,0,12/22/98,12,22,98,22,NaN
121,0,10/02/96,10,02,96,02,NaN
122,0,11/05/90,11,05,90,05,NaN
